In [1]:
!pip install pandas
import pandas as pd
!pip install numpy
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
!pip install matplotlib
import matplotlib.pyplot as plt
import nibabel as nib
import keras
from keras.layers import Input
from tensorflow.keras.applications import ResNet50
!pip install scikit-image
import skimage.transform as skTrans

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


2023-10-23 16:51:52.799557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 16:51:53.237878: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-23 16:51:53.237921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-23 16:51:53.237926: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [2]:
def path_data_axial(category,patient):
    path = "./data/images/"+category+"/axial/"+patient+".png"
    return path
def path_data_sagital(category,patient):
    path = "./data/images/"+category+"/sagital/"+patient+".png"
    return path
def path_data_coronal(category,patient):
    path = "./data/images/"+category+"/coronal/"+patient+".png"
    return path

In [3]:
#reading datas

# total_data = pd.read_csv("./total_data_finally.csv")
# PID = total_data["Subject ID"]
# Labels = total_data["Research Group"]
# total_data
total_data = pd.read_csv("./total_data.csv")
total_data = total_data[(total_data["Research Group"] != "MCI")]
total_data = total_data.reset_index()
PID = total_data["Subject ID"]
Labels = total_data["Research Group"]

In [4]:
# paths = []
# for i in range(0,len(PID)):
#     paths.append(path_data(Labels[i],PID[i]))
    
# paths

In [5]:
new_labels = []
for i in Labels:
    if i == "CN":
        new_labels.append(0);
    elif i == "MCI":
        new_labels.append(2);
    else:
        new_labels.append(1); 

In [6]:
data = []# List of tuples (image_path, label)
for i in range(0,len(PID)):
    data.append((PID[i],new_labels[i],Labels[i]))

data

[('002_S_0295', 0, 'CN'),
 ('002_S_0413', 0, 'CN'),
 ('002_S_0559', 0, 'CN'),
 ('002_S_0619', 1, 'AD'),
 ('002_S_0685', 0, 'CN'),
 ('002_S_0816', 1, 'AD'),
 ('002_S_0938', 1, 'AD'),
 ('002_S_0955', 1, 'AD'),
 ('002_S_1018', 1, 'AD'),
 ('002_S_1261', 0, 'CN'),
 ('002_S_1280', 0, 'CN'),
 ('002_S_4213', 0, 'CN'),
 ('002_S_4225', 0, 'CN'),
 ('002_S_4262', 0, 'CN'),
 ('002_S_4264', 0, 'CN'),
 ('002_S_4270', 0, 'CN'),
 ('002_S_5018', 1, 'AD'),
 ('002_S_6053', 0, 'CN'),
 ('003_S_0907', 0, 'CN'),
 ('003_S_0981', 0, 'CN'),
 ('003_S_1021', 0, 'CN'),
 ('003_S_1059', 1, 'AD'),
 ('003_S_1257', 1, 'AD'),
 ('003_S_4081', 0, 'CN'),
 ('003_S_4118', 0, 'CN'),
 ('003_S_4119', 0, 'CN'),
 ('003_S_4136', 1, 'AD'),
 ('003_S_4142', 1, 'AD'),
 ('003_S_4152', 1, 'AD'),
 ('003_S_4288', 0, 'CN'),
 ('003_S_4350', 0, 'CN'),
 ('003_S_4373', 1, 'AD'),
 ('003_S_4441', 0, 'CN'),
 ('003_S_4555', 0, 'CN'),
 ('003_S_4644', 0, 'CN'),
 ('003_S_4839', 0, 'CN'),
 ('003_S_4840', 0, 'CN'),
 ('003_S_4872', 0, 'CN'),
 ('003_S_489

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from PIL import Image
from torch.optim import lr_scheduler

# Define the ResNet-18 model
model = models.resnet18(pretrained=True)
model
# model

/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# Define the data loader and transformations
class ImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path, label = self.data[index]
#         img = Image.open(img_path)
        img = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [9]:
transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
import torch
import torchvision
from torchvision import transforms
from sklearn import svm
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np


# Define the SVM classifier and its hyperparameters
clf_a = svm.SVC(kernel='linear', C=1, gamma='auto',decision_function_shape='ovo')
clf_s = svm.SVC(kernel='linear', C=1, gamma='auto',decision_function_shape='ovo')
clf_c = svm.SVC(kernel='linear', C=1, gamma='auto',decision_function_shape='ovo')


k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True)

# Train the model using k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(data,new_labels)):
    train_data = [data[i] for i in train_idx]
    val_data = [data[j] for j in val_idx]
    #prepare data for sagital coronal and axial
    train_data_s =[]
    train_data_a = []
    train_data_c = []
    for i in train_data:
        train_data_s.append((path_data_sagital(str(i[2]),i[0]),i[1]))
        train_data_a.append((path_data_sagital(str(i[2]),i[0]),i[1]))
        train_data_c.append((path_data_sagital(str(i[2]),i[0]),i[1]))
#     print(train_data_c)
        
    val_data_s =[]
    val_data_a = []
    val_data_c = []
    for i in val_data:
        val_data_s.append((path_data_sagital(str(i[2]),i[0]),i[1]))
        val_data_a.append((path_data_sagital(str(i[2]),i[0]),i[1]))
        val_data_c.append((path_data_sagital(str(i[2]),i[0]),i[1]))
        
#     print(val_data_c)
        
    # Define the data loaders for all
    train_dataset_a = ImageDataset(train_data_a, transform=transform)
    val_dataset_a = ImageDataset(val_data_a, transform=transform)
    train_loader_a = DataLoader(train_dataset_a, batch_size=8, shuffle=True)
    val_loader_a = DataLoader(val_dataset_a, batch_size=8, shuffle=True)
    
    train_dataset_s = ImageDataset(train_data_s, transform=transform)
    val_dataset_s = ImageDataset(val_data_s, transform=transform)
    train_loader_s = DataLoader(train_dataset_s, batch_size=8, shuffle=True)
    val_loader_s = DataLoader(val_dataset_s, batch_size=8, shuffle=True)
    
    train_dataset_c = ImageDataset(train_data_c, transform=transform)
    val_dataset_c = ImageDataset(val_data_c, transform=transform)
    train_loader_c = DataLoader(train_dataset_c, batch_size=8, shuffle=True)
    val_loader_c= DataLoader(val_dataset_c, batch_size=8, shuffle=True)
    
    # Define the model axial
    model_a = torchvision.models.resnet18(pretrained=True)
    num_ftrs = model_a.fc.in_features
    for param in model_a.parameters():
        param.requires_grad = False
    model_a.fc = nn.Sequential(nn.Linear(model.fc.in_features, 150),
                              nn.ReLU(inplace=True))

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
#     optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=0.001)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model on the training set
    # Extract the 128 features from the model for each image in the dataset
    train_features_a = []
    train_labels_a = []
    for epoch in range(30):
        model_a.train()
        for images, labels in train_loader_a:
            optimizer.zero_grad()
            outputs = model_a(images)
            loss = criterion(outputs, labels)
            loss.backward()
            train_features_a.append(outputs.detach().numpy())
            train_labels_a.append(labels.numpy())
            optimizer.step()
            
    train_features_a = np.array(train_features_a)
    train_labels_a = np.array(train_labels_a)
    n,x,y = train_features_a.shape
    train_features_a = train_features_a.reshape(n*x,y)
    n,x = train_labels_a.shape
    train_labels_a = train_labels_a.reshape(n*x)
    
    model_s = torchvision.models.resnet18(pretrained=True)
    num_ftrs = model_s.fc.in_features
    for param in model_s.parameters():
        param.requires_grad = False
    model_s.fc = nn.Sequential(nn.Linear(model.fc.in_features, 150),
                              nn.ReLU(inplace=True))


#  Define the model sagital

    train_features_s = []
    train_labels_s = []
    for epoch in range(30):
        model_s.train()
        for images, labels in train_loader_s:
            optimizer.zero_grad()
            outputs = model_s(images)
            
            loss = criterion(outputs, labels)
            loss.backward()
            train_features_s.append(outputs.detach().numpy())
            train_labels_s.append(labels.numpy())
            optimizer.step()
            
    train_features_s = np.array(train_features_s)
    train_features_s.shape
    train_labels_s = np.array(train_labels_s)
    n,x,y = train_features_s.shape
    train_features_s = train_features_s.reshape(n*x,y)
    n,x = train_labels_s.shape
    train_labels_s = train_labels_s.reshape(n*x)
    
    #Define model for Coronal
    model_c = torchvision.models.resnet18(pretrained=True)
    num_ftrs = model_c.fc.in_features
    for param in model_c.parameters():
        param.requires_grad = False
    model_c.fc = nn.Sequential(nn.Linear(model.fc.in_features, 150),
                              nn.ReLU(inplace=True))




    train_features_c = []
    train_labels_c = []
    for epoch in range(30):
        model_c.train()
        for images, labels in train_loader_c:
            optimizer.zero_grad()
            outputs = model_c(images)
            loss = criterion(outputs, labels)
            loss.backward()
            train_features_c.append(outputs.detach().numpy())
            train_labels_c.append(labels.numpy())
            optimizer.step()
            
    train_features_c = np.array(train_features_c)
    train_features_c.shape
    train_labels_c = np.array(train_labels_c)
    n,x,y = train_features_c.shape
    train_features_c = train_features_c.reshape(n*x,y)
    n,x = train_labels_c.shape
    train_labels_c = train_labels_c.reshape(n*x)


    # Train the SVM classifier on the 128-dimensional feature vectors axial
    clf_a.fit(train_features_a, train_labels_a)
    train_acc_a = clf_a.score(train_features_a, train_labels_a)
    train_f1_a = f1_score(train_labels_a, clf_a.predict(train_features_a), average='macro')
    print("Axial:")
    print(f'Fold {fold+1}, Train Accuracy: {train_acc_a:.2f}%, Train F1 Score: {train_f1_a:.2f}%')
    
    # Train the SVM classifier on the 128-dimensional feature vectors sagital
    clf_s.fit(train_features_s, train_labels_s)
    train_acc_s = clf_s.score(train_features_s, train_labels_s)
    train_f1_s = f1_score(train_labels_s, clf_s.predict(train_features_s), average='macro')
    print("Sagital:")
    print(f'Fold {fold+1}, Train Accuracy: {train_acc_s:.2f}%, Train F1 Score: {train_f1_s:.2f}%')
    
    # Train the SVM classifier on the 128-dimensional feature vectors coronal
    clf_c.fit(train_features_c, train_labels_c)
    train_acc_c = clf_c.score(train_features_c, train_labels_c)
    train_f1_c = f1_score(train_labels_c, clf_c.predict(train_features_c), average='macro')
    print("Coronal:")
    print(f'Fold {fold+1}, Train Accuracy: {train_acc_c:.2f}%, Train F1 Score: {train_f1_c:.2f}%')

    # Evaluate the model on the test set axial
    test_features_a = []
    test_labels_a = []
    for images, labels in val_loader_a:
        outputs = model_a(images)
        test_features_a.extend(outputs.detach().numpy())
        test_labels_a.extend(labels.numpy())
#     print(test_features)
    test_features_a = np.array(test_features_a)
    test_labels_a = np.array(test_labels_a)
    test_acc_a = clf_a.score(test_features_a, test_labels_a)
    test_f1_a = f1_score(test_labels_a, clf_a.predict(test_features_a), average='macro')
    print("Axial")
    print(f'Fold {fold+1}, Test Accuracy: {test_acc_a:.2f}%, Test F1 Score: {test_f1_a:.2}')
    
    # Evaluate the model on the test set sagital
    test_features_s = []
    test_labels_s = []
    for images, labels in val_loader_s:
        outputs = model_s(images)
        test_features_s.extend(outputs.detach().numpy())
        test_labels_s.extend(labels.numpy())
#     print(test_features)
    test_features_s = np.array(test_features_s)
    test_labels_s = np.array(test_labels_s)
    test_acc_s = clf_s.score(test_features_s, test_labels_s)
    test_f1_s = f1_score(test_labels_s, clf_s.predict(test_features_s), average='macro')
    print("Sagital")
    print(f'Fold {fold+1}, Test Accuracy: {test_acc_s:.2f}%, Test F1 Score: {test_f1_s:.2}')
    # Evaluate the model on the test set coronal
    test_features_c = []
    test_labels_c = []
    for images, labels in val_loader_c:
        outputs = model_c(images)
        test_features_c.extend(outputs.detach().numpy())
        test_labels_c.extend(labels.numpy())
#     print(test_features)
    test_features_c = np.array(test_features_c)
    test_labels_c = np.array(test_labels_c)
    test_acc_c = clf_c.score(test_features_c, test_labels_c)
    test_f1_c = f1_score(test_labels_c, clf_c.predict(test_features_c), average='macro')
    print("Coronal")
    print(f'Fold {fold+1}, Test Accuracy: {test_acc_c:.2f}%, Test F1 Score: {test_f1_c:.2}')
    
    
    #total 
    total_predicted = []
    coronal_p = clf_c.predict(test_features_c)
    axial_p = clf_a.predict(test_features_a)
    sagital_p = clf_s.predict(test_features_s)
    
    for i in range(0,len(coronal_p)):
        c = coronal_p[i]
        a = axial_p[i]
        s = sagital_p[i]
        if a==s :
            total_predicted.append(a)
        elif s==c:
            total_predicted.append(s)
        elif c==a:
            total_predicted.append(c)
        else:
            total_predicted.append(a) 
    test_acc_t = accuracy_score(test_labels_c, total_predicted)
    test_f1_t = f1_score(test_labels_c, total_predicted, average='macro')
    print("total")
    print(f'Fold {fold+1}, Test Accuracy: {test_acc_t:.2f}%, Test F1 Score: {test_f1_t:.2}')      
    

/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Axial:
Fold 1, Train Accuracy: 0.73%, Train F1 Score: 0.73%
Sagital:
Fold 1, Train Accuracy: 0.72%, Train F1 Score: 0.72%
Coronal:
Fold 1, Train Accuracy: 0.71%, Train F1 Score: 0.71%
Axial
Fold 1, Test Accuracy: 0.58%, Test F1 Score: 0.58
Sagital
Fold 1, Test Accuracy: 0.60%, Test F1 Score: 0.6
Coronal
Fold 1, Test Accuracy: 0.55%, Test F1 Score: 0.55
total
Fold 1, Test Accuracy: 0.56%, Test F1 Score: 0.56


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Axial:
Fold 2, Train Accuracy: 0.71%, Train F1 Score: 0.71%
Sagital:
Fold 2, Train Accuracy: 0.71%, Train F1 Score: 0.71%
Coronal:
Fold 2, Train Accuracy: 0.72%, Train F1 Score: 0.72%
Axial
Fold 2, Test Accuracy: 0.53%, Test F1 Score: 0.52
Sagital
Fold 2, Test Accuracy: 0.63%, Test F1 Score: 0.63
Coronal
Fold 2, Test Accuracy: 0.57%, Test F1 Score: 0.57
total
Fold 2, Test Accuracy: 0.66%, Test F1 Score: 0.66


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
clf_s.predict(test_features_s)

In [ ]:
test_labels_s